# Importing Data and Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import pickle
from gensim.models import Word2Vec
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df = pd.read_csv('/kaggle/input/question-pairs-dataset/questions.csv')
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404346,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404347,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1
404348,404348,789796,789797,What is one coin?,What's this coin?,0
404349,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [4]:
df = df.dropna(subset=['question1', 'question2'])

In [5]:
df = df[(df['question1'].str.strip() != '') & (df['question2'].str.strip() != '')]

In [6]:
df = df[(df['question1'].str.len() <= 500) & (df['question2'].str.len() <= 500)]

In [7]:
df = df[~df['question1'].str.contains(r'\d') &  ~df['question2'].str.contains(r'\d')]

In [8]:
df = df[df['question1'].apply(lambda x: isinstance(x, str)) &  df['question2'].apply(lambda x: isinstance(x, str))]

In [9]:
df = df.reset_index(drop=True)

In [10]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
4,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
...,...,...,...,...,...,...
340047,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
340048,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1
340049,404348,789796,789797,What is one coin?,What's this coin?,0
340050,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


# Check for Missing Values

In [11]:
df.shape

(340052, 6)

In [12]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [13]:
df = df.dropna()

In [14]:
df.shape

(340052, 6)

## Inspecting Data Uniqueness and Distribution

In [15]:
# Check for duplicate rows
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Check the distribution of the target column `is_duplicate`
print(df['is_duplicate'].value_counts())

Number of duplicate rows: 0
is_duplicate
0    210239
1    129813
Name: count, dtype: int64


# Checking for Class Balance

In [16]:
duplicate_percentage = df['is_duplicate'].mean() * 100
print(f"Percentage of duplicate pairs: {duplicate_percentage:.2f}%")
print(f"Percentage of non-duplicate pairs: {100 - duplicate_percentage:.2f}%")

Percentage of duplicate pairs: 38.17%
Percentage of non-duplicate pairs: 61.83%


# Remove Columns

In [17]:
# Remove the 'id', 'qid1', and 'qid2' columns
df = df.drop(['id', 'qid1', 'qid2'], axis=1)

# Display the updated DataFrame
df.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
4,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1


# Text Preprocessing

In [18]:
def preprocess_text(text):
    # Check for NaN values and return an empty string or the original text
    if pd.isna(text):
        return ''

    # Lowercasing
    text = text.lower()

    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Removing numbers
    text = re.sub(r'\d+', '', text)

    # Removing special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Removing extra whitespaces
    text = ' '.join(tokens)

    return text

In [19]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [20]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [21]:
# Preprocess the questions
df['question1'] = df['question1'].apply(preprocess_text)
df['question2'] = df['question2'].apply(preprocess_text)

# Display the updated DataFrame
df[['question1', 'question2']].head()

,question1,question2
0,step step guide invest share market india,step step guide invest share market
1,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...
2,increase speed internet connection using vpn,internet speed increased hacking dns
3,one dissolve water quikly sugar salt methane c...,fish would survive salt water
4,astrology capricorn sun cap moon cap risingwha...,im triple capricorn sun moon ascendant caprico...


In [22]:
df

,question1,question2,is_duplicate
0,step step guide invest share market india,step step guide invest share market,0
1,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,0
2,increase speed internet connection using vpn,internet speed increased hacking dns,0
3,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0
4,astrology capricorn sun cap moon cap risingwha...,im triple capricorn sun moon ascendant caprico...,1
...,...,...,...
340047,many keywords racket programming language late...,many keywords perl programming language latest...,0
340048,believe life death,true life death,1
340049,one coin,whats coin,0
340050,approx annual cost living studying uic chicago...,little hairfall problem want use hair styling ...,0


# Feature Engineering

In [23]:
# Word Overlap Features
def jaccard_similarity(q1, q2):
    a = set(q1.split())
    b = set(q2.split())

    if len(a) == 0 and len(b) == 0:
        return 1.0
    elif len(a) == 0 or len(b) == 0:
        return 0.0

    return len(a.intersection(b)) / len(a.union(b))

In [24]:
df['jaccard'] = df.apply(lambda row: jaccard_similarity(row['question1'], row['question2']), axis=1)

In [25]:
df

,question1,question2,is_duplicate,jaccard
0,step step guide invest share market india,step step guide invest share market,0,0.833333
1,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,0,0.222222
2,increase speed internet connection using vpn,internet speed increased hacking dns,0,0.222222
3,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0,0.153846
4,astrology capricorn sun cap moon cap risingwha...,im triple capricorn sun moon ascendant caprico...,1,0.400000
...,...,...,...,...
340047,many keywords racket programming language late...,many keywords perl programming language latest...,0,0.750000
340048,believe life death,true life death,1,0.500000
340049,one coin,whats coin,0,0.333333
340050,approx annual cost living studying uic chicago...,little hairfall problem want use hair styling ...,0,0.000000


In [26]:
def compute_token_features(row):
    q1 = row['question1'].split()
    q2 = row['question2'].split()

    # Convert to sets for comparison
    tokens_q1 = set(q1)
    tokens_q2 = set(q2)

    # Count common words
    cwc = len(tokens_q1.intersection(tokens_q2))

    # Count common stop words
    csc = len(tokens_q1.intersection(tokens_q2).intersection(stop_words))

    # Length of questions
    len_q1, len_q2 = len(tokens_q1), len(tokens_q2)

    # Token Features
    cwc_min = cwc / min(len_q1, len_q2) if min(len_q1, len_q2) > 0 else 0
    cwc_max = cwc / max(len_q1, len_q2) if max(len_q1, len_q2) > 0 else 0

    csc_min = csc / min(len([word for word in tokens_q1 if word in stop_words]),
                         len([word for word in tokens_q2 if word in stop_words])) if min(len([word for word in tokens_q1 if word in stop_words]),
                         len([word for word in tokens_q2 if word in stop_words])) > 0 else 0

    csc_max = csc / max(len([word for word in tokens_q1 if word in stop_words]),
                         len([word for word in tokens_q2 if word in stop_words])) if max(len([word for word in tokens_q1 if word in stop_words]),
                         len([word for word in tokens_q2 if word in stop_words])) > 0 else 0

    # Common Tokens
    ctc_min = cwc / min(len_q1, len_q2) if min(len_q1, len_q2) > 0 else 0
    ctc_max = cwc / max(len_q1, len_q2) if max(len_q1, len_q2) > 0 else 0

    # Last and first word equality
    last_word_eq = int(q1[-1] == q2[-1]) if q1 and q2 else 0
    first_word_eq = int(q1[0] == q2[0]) if q1 and q2 else 0

    return pd.Series([cwc_min, cwc_max, csc_min, csc_max, ctc_min, ctc_max, last_word_eq, first_word_eq])

In [27]:
def compute_length_features(row):
    len_q1 = len(row['question1'].split())
    len_q2 = len(row['question2'].split())

    mean_len = (len_q1 + len_q2) / 2
    abs_len_diff = abs(len_q1 - len_q2)

    # Finding longest common substring ratio
    def longest_common_substring_ratio(s1, s2):
        # Find longest common substring using dynamic programming
        max_length = 0
        for i in range(len(s1)):
            for j in range(len(s2)):
                length = 0
                while (i + length < len(s1)) and (j + length < len(s2)) and (s1[i + length] == s2[j + length]):
                    length += 1
                max_length = max(max_length, length)
        return max_length / min(len(s1), len(s2)) if min(len(s1), len(s2)) > 0 else 0

    longest_substr_ratio = longest_common_substring_ratio(row['question1'], row['question2'])

    return pd.Series([mean_len, abs_len_diff, longest_substr_ratio])

In [28]:
def compute_fuzzy_features(row):
    q1, q2 = row['question1'], row['question2']

    fuzz_ratio = fuzz.ratio(q1, q2)
    fuzz_partial_ratio = fuzz.partial_ratio(q1, q2)
    token_sort_ratio = fuzz.token_sort_ratio(q1, q2)
    token_set_ratio = fuzz.token_set_ratio(q1, q2)

    return pd.Series([fuzz_ratio, fuzz_partial_ratio, token_sort_ratio, token_set_ratio])

In [29]:
stop_words = set(stopwords.words('english'))

In [30]:
token_features = df[['question1', 'question2']].apply(compute_token_features, axis=1)
token_features.columns = ['cwc_min', 'cwc_max', 'csc_min', 'csc_max', 'ctc_min', 'ctc_max', 'last_word_eq', 'first_word_eq']

In [31]:
token_features

,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq
0,1.000000,0.833333,0.0,0.0,1.000000,0.833333,0.0,1.0
1,0.666667,0.250000,0.0,0.0,0.666667,0.250000,0.0,0.0
2,0.400000,0.333333,0.0,0.0,0.400000,0.333333,0.0,0.0
3,0.400000,0.200000,0.0,0.0,0.400000,0.200000,0.0,0.0
4,0.571429,0.571429,0.0,0.0,0.571429,0.571429,1.0,0.0
...,...,...,...,...,...,...,...,...
340047,0.857143,0.857143,0.0,0.0,0.857143,0.857143,1.0,1.0
340048,0.666667,0.666667,0.0,0.0,0.666667,0.666667,1.0,0.0
340049,0.500000,0.500000,0.0,0.0,0.500000,0.500000,1.0,0.0
340050,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [32]:
length_features = df.apply(compute_length_features, axis=1)
length_features.columns = ['mean_len', 'abs_len_diff', 'longest_substr_ratio']

In [33]:
length_features

,mean_len,abs_len_diff,longest_substr_ratio
0,6.5,1.0,1.000000
1,6.5,5.0,0.838710
2,5.5,1.0,0.250000
3,7.5,5.0,0.206897
4,8.0,0.0,0.294118
...,...,...,...
340047,7.0,0.0,0.666667
340048,3.0,0.0,0.800000
340049,2.0,0.0,0.625000
340050,11.0,4.0,0.065574


In [34]:
fuzzy_features = df.apply(compute_fuzzy_features, axis=1)
fuzzy_features.columns = ['fuzz_ratio', 'fuzz_partial_ratio', 'token_sort_ratio', 'token_set_ratio']

In [35]:
fuzzy_features

,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,92,100,92,100
1,59,94,59,84
2,55,46,65,65
3,25,52,40,51
4,64,65,50,69
...,...,...,...,...
340047,93,91,89,95
340048,73,80,61,80
340049,56,62,56,67
340050,35,38,29,29


In [36]:
df = pd.concat([df, token_features, length_features, fuzzy_features], axis=1)

In [37]:
df

,question1,question2,is_duplicate,jaccard,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,mean_len,abs_len_diff,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
0,step step guide invest share market india,step step guide invest share market,0,0.833333,1.000000,0.833333,0.0,0.0,1.000000,0.833333,0.0,1.0,6.5,1.0,1.000000,92,100,92,100
1,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,0,0.222222,0.666667,0.250000,0.0,0.0,0.666667,0.250000,0.0,0.0,6.5,5.0,0.838710,59,94,59,84
2,increase speed internet connection using vpn,internet speed increased hacking dns,0,0.222222,0.400000,0.333333,0.0,0.0,0.400000,0.333333,0.0,0.0,5.5,1.0,0.250000,55,46,65,65
3,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0,0.153846,0.400000,0.200000,0.0,0.0,0.400000,0.200000,0.0,0.0,7.5,5.0,0.206897,25,52,40,51
4,astrology capricorn sun cap moon cap risingwha...,im triple capricorn sun moon ascendant caprico...,1,0.400000,0.571429,0.571429,0.0,0.0,0.571429,0.571429,1.0,0.0,8.0,0.0,0.294118,64,65,50,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340047,many keywords racket programming language late...,many keywords perl programming language latest...,0,0.750000,0.857143,0.857143,0.0,0.0,0.857143,0.857143,1.0,1.0,7.0,0.0,0.666667,93,91,89,95
340048,believe life death,true life death,1,0.500000,0.666667,0.666667,0.0,0.0,0.666667,0.666667,1.0,0.0,3.0,0.0,0.800000,73,80,61,80
340049,one coin,whats coin,0,0.333333,0.500000,0.500000,0.0,0.0,0.500000,0.500000,1.0,0.0,2.0,0.0,0.625000,56,62,56,67
340050,approx annual cost living studying uic chicago...,little hairfall problem want use hair styling ...,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,11.0,4.0,0.065574,35,38,29,29


In [38]:
# Combine question1 and question2 into a single list for TF-IDF
questions_combined = df['question1'].tolist() + df['question2'].tolist()

In [39]:
questions_combined

['step step guide invest share market india',
 'story kohinoor kohinoor diamond',
 'increase speed internet connection using vpn',
 'one dissolve water quikly sugar salt methane carbon di oxide',
 'astrology capricorn sun cap moon cap risingwhat say',
 'buy tiago',
 'good geologist',
 'use instead',
 'method find separation slit using fresnel biprism',
 'read find youtube comment',
 'make physic easy learn',
 'first sexual experience like',
 'law change status student visa green card u compare immigration law canada',
 'manipulation mean',
 'girl want friend guy reject',
 'many quora user posting question readily answered google',
 'best digital marketing institution banglore',
 'rocket look white',
 'whats causing someone jealous',
 'question ask quora',
 'mean every time look clock number',
 'tip making job interview process medicine',
 'web application',
 'society place much importance sport',
 'best way make money online',
 'prepare ca final law',
 'whats one thing would like bette

# Add TF-IDF Features

In [40]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.95, max_features=500)

In [41]:
# Fit and transform the combined questions
tfidf_matrix = tfidf_vectorizer.fit_transform(questions_combined)

In [42]:
# Save TF-IDF model
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

In [43]:
tfidf_matrix.shape

(680104, 500)

In [44]:
# Convert the sparse matrix to a DataFrame
tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf_matrix, columns=tfidf_vectorizer.get_feature_names_out())

In [45]:
num_questions = len(df)
tfidf_question1 = tfidf_matrix[:num_questions, :]
tfidf_question2 = tfidf_matrix[num_questions:, :]

In [46]:
tfidf_question1_df = pd.DataFrame.sparse.from_spmatrix(tfidf_question1)
tfidf_question2_df = pd.DataFrame.sparse.from_spmatrix(tfidf_question2)

In [47]:
tfidf_question1_df.columns = [f'tfidf_q1_{i}' for i in range(tfidf_question1_df.shape[1])]
tfidf_question2_df.columns = [f'tfidf_q2_{i}' for i in range(tfidf_question2_df.shape[1])]

In [48]:
tfidf_question1_df

,tfidf_q1_0,tfidf_q1_1,tfidf_q1_2,tfidf_q1_3,tfidf_q1_4,tfidf_q1_5,tfidf_q1_6,tfidf_q1_7,tfidf_q1_8,tfidf_q1_9,...,tfidf_q1_490,tfidf_q1_491,tfidf_q1_492,tfidf_q1_493,tfidf_q1_494,tfidf_q1_495,tfidf_q1_496,tfidf_q1_497,tfidf_q1_498,tfidf_q1_499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340047,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
340048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
340049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
340050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
tfidf_question2_df

,tfidf_q2_0,tfidf_q2_1,tfidf_q2_2,tfidf_q2_3,tfidf_q2_4,tfidf_q2_5,tfidf_q2_6,tfidf_q2_7,tfidf_q2_8,tfidf_q2_9,...,tfidf_q2_490,tfidf_q2_491,tfidf_q2_492,tfidf_q2_493,tfidf_q2_494,tfidf_q2_495,tfidf_q2_496,tfidf_q2_497,tfidf_q2_498,tfidf_q2_499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.355537,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.588985,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340047,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
340048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
340049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
340050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
df = pd.concat([df, tfidf_question1_df, tfidf_question2_df], axis=1)

In [51]:
df

,question1,question2,is_duplicate,jaccard,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,...,tfidf_q2_490,tfidf_q2_491,tfidf_q2_492,tfidf_q2_493,tfidf_q2_494,tfidf_q2_495,tfidf_q2_496,tfidf_q2_497,tfidf_q2_498,tfidf_q2_499
0,step step guide invest share market india,step step guide invest share market,0,0.833333,1.000000,0.833333,0.0,0.0,1.000000,0.833333,...,0,0,0,0,0,0,0,0,0,0
1,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,0,0.222222,0.666667,0.250000,0.0,0.0,0.666667,0.250000,...,0,0,0,0.355537,0,0,0,0,0,0
2,increase speed internet connection using vpn,internet speed increased hacking dns,0,0.222222,0.400000,0.333333,0.0,0.0,0.400000,0.333333,...,0,0,0,0,0,0,0,0,0,0
3,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0,0.153846,0.400000,0.200000,0.0,0.0,0.400000,0.200000,...,0,0,0,0.588985,0,0,0,0,0,0
4,astrology capricorn sun cap moon cap risingwha...,im triple capricorn sun moon ascendant caprico...,1,0.400000,0.571429,0.571429,0.0,0.0,0.571429,0.571429,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340047,many keywords racket programming language late...,many keywords perl programming language latest...,0,0.750000,0.857143,0.857143,0.0,0.0,0.857143,0.857143,...,0,0,0,0,0,0,0,0,0,0
340048,believe life death,true life death,1,0.500000,0.666667,0.666667,0.0,0.0,0.666667,0.666667,...,0,0,0,0,0,0,0,0,0,0
340049,one coin,whats coin,0,0.333333,0.500000,0.500000,0.0,0.0,0.500000,0.500000,...,0,0,0,0,0,0,0,0,0,0
340050,approx annual cost living studying uic chicago...,little hairfall problem want use hair styling ...,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0


# Word2Vec

In [52]:
# Preprocess text: tokenization and lowercasing
def preprocess(text):
    text = str(text)
    return text.lower().split()

In [53]:
# Tokenize the questions
df['tokens_q1'] = df['question1'].apply(preprocess)
df['tokens_q2'] = df['question2'].apply(preprocess)

In [54]:
# Combine tokens from both questions
all_tokens = df['tokens_q1'].tolist() + df['tokens_q2'].tolist()

In [55]:
# Train Word2Vec model
model_word2vec = Word2Vec(sentences=all_tokens, vector_size=300, window=5, min_count=1, workers=5)

In [56]:
# Save Word2Vec model
model_word2vec.save('word2vec_model.model')

In [57]:
# Function to compute average word vectors for a given list of tokens
def get_vector(tokens):
    vectors = [model_word2vec.wv[word] for word in tokens if word in model_word2vec.wv]
    if not vectors:
        return np.zeros(model_word2vec.vector_size)  # Return zero vector if no words are in the model
    return np.mean(vectors, axis=0)

In [58]:
# Get average vectors for question1 and question2
df['w2v_q1'] = df['tokens_q1'].apply(get_vector)
df['w2v_q2'] = df['tokens_q2'].apply(get_vector)

In [59]:
# Convert list of vectors into a array
w2v_q1_arr = np.array(df['w2v_q1'].tolist())
w2v_q2_arr = np.array(df['w2v_q2'].tolist())

In [60]:
# Create DataFrames from NumPy arrays
w2v_df_q1 = pd.DataFrame(w2v_q1_arr, columns=[f'w2v_q1_{i}' for i in range(w2v_q1_arr.shape[1])])
w2v_df_q2 = pd.DataFrame(w2v_q2_arr, columns=[f'w2v_q2_{i}' for i in range(w2v_q2_arr.shape[1])])

In [61]:
w2v_df_q1

,w2v_q1_0,w2v_q1_1,w2v_q1_2,w2v_q1_3,w2v_q1_4,w2v_q1_5,w2v_q1_6,w2v_q1_7,w2v_q1_8,w2v_q1_9,...,w2v_q1_290,w2v_q1_291,w2v_q1_292,w2v_q1_293,w2v_q1_294,w2v_q1_295,w2v_q1_296,w2v_q1_297,w2v_q1_298,w2v_q1_299
0,0.040392,0.414507,-0.056899,0.126704,0.208801,-0.699201,0.162275,0.435012,-0.396042,-0.243946,...,-0.301271,-0.007709,0.352584,0.022273,0.032988,0.097632,-0.291257,0.850012,-1.057755,0.133349
1,0.240623,0.061684,-0.213581,0.025495,-0.197190,-0.429052,0.355306,0.106072,-0.035058,-0.451976,...,0.089028,0.162852,0.057580,0.046787,0.266077,0.263316,-0.040569,0.480197,-0.019062,-0.197928
2,-0.299933,0.196113,0.442500,0.832265,-0.105008,-0.033562,0.061597,0.869867,-0.799992,0.260121,...,-0.539593,-0.297501,-0.204776,-0.379759,-0.477616,0.410480,0.116658,-0.103035,0.408778,-0.005818
3,0.141519,0.565261,0.116938,0.550075,-0.073917,-0.432158,0.235965,0.409277,0.038234,0.176196,...,-0.090171,0.020927,-0.054936,-0.377711,-0.135841,0.103885,0.062311,-0.397433,0.047688,-0.120842
4,-0.069389,0.073142,0.153375,-0.057709,0.200283,-0.089484,0.476654,0.526427,-0.054898,-0.273758,...,-0.170733,-0.014073,-0.187021,0.043699,0.193394,0.060553,-0.166477,-0.140764,-0.040809,-0.010197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340047,-0.084776,0.314526,-0.426070,-0.667263,-0.329587,0.017644,0.107760,0.181881,0.310411,-0.207456,...,-0.464944,0.337506,0.236243,0.072974,-0.126935,0.305871,0.099825,-0.117730,0.314348,-0.044186
340048,0.840465,-0.001050,0.080234,0.442124,0.099042,0.214769,0.404373,1.135080,-0.639298,-0.197870,...,-0.250920,0.338639,-0.159746,-0.165318,-0.442076,0.280600,0.135573,-0.108485,-0.170422,-0.539849
340049,0.321438,-0.166370,-0.250907,-0.140746,-0.262915,-0.125416,0.569728,-0.017146,-0.139360,0.469214,...,0.003684,-0.001931,0.336685,0.225963,-0.268451,-0.098227,0.624040,-0.367315,-0.020808,-0.049922
340050,0.242128,-0.310800,-0.255888,-0.195377,-0.099839,-0.277696,0.069680,0.352217,0.257178,0.158341,...,0.030872,0.169288,0.638905,-0.352774,0.360413,-0.116463,-0.358840,-0.114059,-0.540961,0.064872


In [62]:
w2v_df_q2

,w2v_q2_0,w2v_q2_1,w2v_q2_2,w2v_q2_3,w2v_q2_4,w2v_q2_5,w2v_q2_6,w2v_q2_7,w2v_q2_8,w2v_q2_9,...,w2v_q2_290,w2v_q2_291,w2v_q2_292,w2v_q2_293,w2v_q2_294,w2v_q2_295,w2v_q2_296,w2v_q2_297,w2v_q2_298,w2v_q2_299
0,0.036651,0.484858,0.000352,0.090832,0.237110,-0.725121,0.088124,0.506529,-0.246288,-0.243958,...,-0.383143,0.017399,0.304356,0.094833,-0.034310,0.283884,-0.230006,0.867045,-0.982339,0.126760
1,0.240961,0.142455,0.400580,0.361343,-0.058010,-0.217568,0.540772,0.116027,-0.003704,0.356733,...,-0.091704,0.266264,0.001536,-0.020417,-0.048224,-0.271963,0.024115,0.292765,-0.211841,0.138885
2,-0.219854,0.094188,0.079937,0.662037,-0.031476,-0.157401,-0.065556,0.714908,-0.546158,0.321569,...,-0.509510,-0.072489,-0.123812,-0.276580,-0.287965,0.611023,0.026311,-0.217743,0.401819,0.077512
3,0.440132,0.753538,0.430695,1.002406,0.031498,-0.787019,0.139682,0.426566,0.078469,0.641918,...,0.119881,-0.049199,-0.208786,-0.851665,-0.135088,0.058294,0.005978,-0.133319,-0.219183,0.000543
4,-0.073683,-0.278065,0.244538,-0.168462,0.342094,0.168267,0.498928,0.322238,-0.087321,-0.215254,...,-0.142535,0.111944,-0.136529,0.087735,0.457494,0.031340,-0.296608,-0.003721,0.040110,0.004189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340047,-0.077048,0.316825,-0.409524,-0.680792,-0.313299,0.020087,0.131196,0.168598,0.309195,-0.194926,...,-0.474278,0.341189,0.232688,0.076092,-0.157939,0.330268,0.107441,-0.121768,0.324888,-0.043617
340048,0.858027,-0.087757,0.165333,0.565648,0.010704,-0.056164,0.570982,0.908214,-0.537823,-0.189565,...,-0.001726,0.400261,0.092657,0.196068,-0.276760,0.360050,0.248863,-0.184772,-0.198342,-0.259534
340049,-0.452475,0.585455,0.295865,-0.196729,-0.336156,-0.501255,0.596549,0.627137,-0.254467,-0.268732,...,0.103951,0.083543,0.352414,-0.126633,0.248377,0.577341,-0.411738,-0.023730,0.239713,-0.263194
340050,0.108463,0.023620,-0.124778,0.045420,-0.021187,0.075394,0.197883,0.547083,0.120214,-0.167526,...,-0.137789,0.089656,0.073491,-0.141397,0.036171,0.014345,0.108459,0.164505,-0.001216,0.093021


In [63]:
# Combine the new features back into the original DataFrame
df = pd.concat([df, w2v_df_q1, w2v_df_q2], axis=1)

In [64]:
# Drop intermediate columns and original questions
df.drop(columns=['tokens_q1', 'tokens_q2', 'w2v_q1', 'w2v_q2', 'question1', 'question2'], inplace=True)

In [65]:
df

,is_duplicate,jaccard,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,...,w2v_q2_290,w2v_q2_291,w2v_q2_292,w2v_q2_293,w2v_q2_294,w2v_q2_295,w2v_q2_296,w2v_q2_297,w2v_q2_298,w2v_q2_299
0,0,0.833333,1.000000,0.833333,0.0,0.0,1.000000,0.833333,0.0,1.0,...,-0.383143,0.017399,0.304356,0.094833,-0.034310,0.283884,-0.230006,0.867045,-0.982339,0.126760
1,0,0.222222,0.666667,0.250000,0.0,0.0,0.666667,0.250000,0.0,0.0,...,-0.091704,0.266264,0.001536,-0.020417,-0.048224,-0.271963,0.024115,0.292765,-0.211841,0.138885
2,0,0.222222,0.400000,0.333333,0.0,0.0,0.400000,0.333333,0.0,0.0,...,-0.509510,-0.072489,-0.123812,-0.276580,-0.287965,0.611023,0.026311,-0.217743,0.401819,0.077512
3,0,0.153846,0.400000,0.200000,0.0,0.0,0.400000,0.200000,0.0,0.0,...,0.119881,-0.049199,-0.208786,-0.851665,-0.135088,0.058294,0.005978,-0.133319,-0.219183,0.000543
4,1,0.400000,0.571429,0.571429,0.0,0.0,0.571429,0.571429,1.0,0.0,...,-0.142535,0.111944,-0.136529,0.087735,0.457494,0.031340,-0.296608,-0.003721,0.040110,0.004189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340047,0,0.750000,0.857143,0.857143,0.0,0.0,0.857143,0.857143,1.0,1.0,...,-0.474278,0.341189,0.232688,0.076092,-0.157939,0.330268,0.107441,-0.121768,0.324888,-0.043617
340048,1,0.500000,0.666667,0.666667,0.0,0.0,0.666667,0.666667,1.0,0.0,...,-0.001726,0.400261,0.092657,0.196068,-0.276760,0.360050,0.248863,-0.184772,-0.198342,-0.259534
340049,0,0.333333,0.500000,0.500000,0.0,0.0,0.500000,0.500000,1.0,0.0,...,0.103951,0.083543,0.352414,-0.126633,0.248377,0.577341,-0.411738,-0.023730,0.239713,-0.263194
340050,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,-0.137789,0.089656,0.073491,-0.141397,0.036171,0.014345,0.108459,0.164505,-0.001216,0.093021


# Split Data (80-20)

In [66]:
X = df.drop(columns=['is_duplicate'])
y = df['is_duplicate']

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
X_train.shape

(272041, 1616)

In [69]:
X_test.shape

(68011, 1616)

In [70]:
y_train.shape

(272041,)

In [71]:
y_test.shape

(68011,)

# DL Modles

# Model 1: Recurrent Neural Network (RNN) with LSTM


In [72]:
# Convert to dense array
X_train_dense = X_train.values
X_test_dense = X_test.values

In [73]:
X_train_dense

array([[ 0.38461538,  0.625     ,  0.5       , ..., -0.14789167,
        -0.27446395,  0.32398218],
       [ 0.25      ,  0.4       ,  0.4       , ..., -0.51944816,
        -0.00481398,  0.28342372],
       [ 0.33333333,  0.66666667,  0.4       , ...,  0.11060474,
         0.40250373,  0.07572896],
       ...,
       [ 0.33333333,  0.5       ,  0.5       , ..., -0.22565831,
         0.42862701, -0.05598021],
       [ 0.42857143,  0.6       ,  0.6       , ...,  0.95692408,
        -0.10527532, -0.04396657],
       [ 0.75      ,  0.85714286,  0.85714286, ...,  0.98854005,
         0.21993852,  0.7845993 ]])

In [74]:
X_test_dense

array([[ 0.8       ,  1.        ,  0.8       , ...,  0.54363358,
         1.05446172, -0.98739237],
       [ 0.6       ,  0.75      ,  0.75      , ...,  0.48107183,
         0.00781591, -0.18485819],
       [ 0.375     ,  0.6       ,  0.5       , ...,  0.44039169,
        -0.36915737, -0.25446758],
       ...,
       [ 0.25      ,  0.66666667,  0.28571429, ...,  0.11853746,
         0.02497572, -0.16092275],
       [ 0.33333333,  0.66666667,  0.4       , ..., -0.25552896,
         0.07489666,  0.27021563],
       [ 0.33333333,  0.5       ,  0.5       , ...,  0.52711856,
        -0.35812709, -0.03667754]])

In [75]:
# Define the RNN model
rnn_model = Sequential()
rnn_model.add(LSTM(128, input_shape=(X_train_dense.shape[1], 1), return_sequences=True))  # First LSTM layer
rnn_model.add(Bidirectional(LSTM(64, return_sequences=False)))  # Second LSTM layer
rnn_model.add(Dense(32, activation='relu'))  # Fully connected layer
rnn_model.add(Dropout(0.5))  # Dropout layer for regularization
rnn_model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
rnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1616, 128)      │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 169,537 (662.25 KB)

 Trainable params: 169,537 (662.25 KB)

 Non-trainable params: 0 (0.00 B)

In [77]:
batch_size = 32
epochs = 10      

In [78]:
# Training the model
rnn_model.fit(X_train_dense, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2)

Epoch 1/10
6801/6801 - 1407s - 207ms/step - accuracy: 0.7027 - loss: 0.5248 - val_accuracy: 0.7286 - val_loss: 0.4956
Epoch 2/10
6801/6801 - 1418s - 208ms/step - accuracy: 0.7308 - loss: 0.5005 - val_accuracy: 0.7296 - val_loss: 0.5021
Epoch 3/10
6801/6801 - 1417s - 208ms/step - accuracy: 0.7329 - loss: 0.4989 - val_accuracy: 0.7328 - val_loss: 0.4912
Epoch 4/10
6801/6801 - 1420s - 209ms/step - accuracy: 0.7308 - loss: 0.4985 - val_accuracy: 0.7300 - val_loss: 0.4956
Epoch 5/10
6801/6801 - 1417s - 208ms/step - accuracy: 0.7349 - loss: 0.4945 - val_accuracy: 0.7366 - val_loss: 0.4885
Epoch 6/10
6801/6801 - 1418s - 209ms/step - accuracy: 0.7452 - loss: 0.4826 - val_accuracy: 0.7542 - val_loss: 0.4648
Epoch 7/10
6801/6801 - 1415s - 208ms/step - accuracy: 0.7592 - loss: 0.4660 - val_accuracy: 0.7623 - val_loss: 0.4559
Epoch 8/10
6801/6801 - 1419s - 209ms/step - accuracy: 0.7648 - loss: 0.4567 - val_accuracy: 0.7615 - val_loss: 0.4574
Epoch 9/10
6801/6801 - 1425s - 210ms/step - accuracy: 0.

In [79]:
# Making predictions
y_pred_rnn = rnn_model.predict(X_test_dense)
y_pred_rnn_binary = [1 if pred >= 0.5 else 0 for pred in y_pred_rnn]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_rnn_binary)
print("RNN (LSTM) Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred_rnn_binary))

2126/2126 ━━━━━━━━━━━━━━━━━━━━ 194s 91ms/step
RNN (LSTM) Accuracy: 0.7763891135257532
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.80      0.82     42043
           1       0.69      0.74      0.72     25968

    accuracy                           0.78     68011
   macro avg       0.76      0.77      0.77     68011
weighted avg       0.78      0.78      0.78     68011



In [81]:
# Save the model
rnn_model.save('rnn_model.h5')